In [1]:
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

from utils.get_raw_data import locate_raw_data, get_multiple_data_arrays

In [5]:
# expected plot from theory, i.e. only lines
disp =  0.0100283 # keV / channel
offs = 21.078

ch = np.arange(0, 2048)
e = disp * (ch - offs)

def ch_to_e(ch):
    return disp * (ch - offs)

s = get_multiple_data_arrays(filters=['GaAs_30'], normalize=False)[0][1]
m = s.max()
sn = s/m
peaks, dic = find_peaks(sn, prominence=0.01, height=0.01)
pm = s[peaks]

In [13]:
fig = go.Figure()
for p in range(1, len(peaks)):
    fig.add_trace(go.Scatter(x=[ch_to_e(peaks[p]), ch_to_e(peaks[p])], y=[0, pm[p]], mode='lines', name=f'{ch_to_e(peaks[p]):.2f} keV', line=dict(width=5)))
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.update_layout(width=800, height=400)
fig.update_layout(yaxis_range=[-100,75000], xaxis_range=[0.11,12], xaxis_title="Energy [keV]", yaxis_title="Intensity [counts]")
fig.update_layout(legend=dict(x=.85, y=.95, font=dict(family="sans-serif", size=12, color="black"), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=2))

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=14, color="black"))
fig.write_image("../plots/expected_lines.png", scale=2)


In [46]:
# actual plot from data

fig = go.Figure()
fig.add_trace(go.Scatter(x=e, y=s, mode='lines', name='Actual data', line=dict(width=4)))
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.update_layout(width=800, height=400)
fig.update_layout(yaxis_range=[-100,75000], xaxis_range=[0.11,12], xaxis_title="Energy [keV]", yaxis_title="Intensity [counts]")
fig.update_layout(legend=dict(x=.85, y=.95, traceorder="normal", font=dict(family="sans-serif", size=12, color="black"), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=2))
fig.update_layout(legend=dict(x=.85, y=.95, font=dict(family="sans-serif", size=12, color="black"), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=2))
fig.update_layout(showlegend=True)

# annotate "Peaks broaden at higher peaks"
fig.add_annotation(x=0.5, y=0.5, xref="paper", yref="paper", text="Broadening of peaks at higher energy", showarrow=False, font=dict(size=14))

fig.add_annotation(x=0.9, y=35500, xref="x", yref="y", text="→", showarrow=False, font=dict(size=18))
fig.add_annotation(x=1.31, y=35500, xref="x", yref="y", text="←", showarrow=False, font=dict(size=18))

fig.add_annotation(x=9.00, y=25760/2, xref="x", yref="y", text="→", showarrow=False, font=dict(size=18))
fig.add_annotation(x=9.51, y=25760/2, xref="x", yref="y", text="←", showarrow=False, font=dict(size=18))

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=14, color="black"))
fig.write_image("../plots/actual_peaks.png", scale=2)


In [47]:
# background shape

# data where the peaks are set to np.nan
bg = s.copy()

def remove_peaks(data, peaks, buffer=45):
    for i in range(len(peaks)):
        data[int((peaks[i] - buffer)) : int((peaks[i] + buffer))] = np.nan
    return data

bg = remove_peaks(bg, peaks)
bg[194:210] = np.nan
bg[120:210] = np.linspace(0, bg[210], 210-120)
bg[890:1240] = np.linspace(bg[890], bg[1240], 1240-890)
bg = bg[120:]

bg_m = bg.max()
bg = bg/bg_m

# polyfit bg
bg_fit = np.polyfit(e[120:], bg, 6)
bg_fit = np.poly1d(bg_fit)
bg_fit = bg_fit(e)

bg_fit=bg_fit*bg_m

fig = go.Figure()
# fig.add_trace(go.Scatter(x=e, y=bg, mode="lines"))
fig.add_trace(go.Scatter(x=e[30:], y=bg_fit[120:1700], mode="lines", name='Expected background', line=dict(width=4)))
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.update_layout(width=800, height=400)
fig.update_layout(xaxis_title="Energy [keV]", yaxis_title="Intensity [counts]", yaxis_range=[-10,1300])#, xaxis_range=[0.11,12])
fig.update_layout(legend=dict(x=.75, y=.95, traceorder="normal", font=dict(family="sans-serif", size=12, color="black"), bgcolor="LightSteelBlue", bordercolor="Black", borderwidth=2))
fig.update_layout(showlegend=True)

fig.add_annotation(x=0.1, y=0.15, xref="paper", yref="paper", text="Background radiation is absorbed below ~2 keV ", showarrow=False, font=dict(size=14))
fig.add_annotation(x=0.6, y=0.6, xref="paper", yref="paper", text="Background radiation decrease exponentially", showarrow=False, font=dict(size=14))

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=14, color="black"))
# save as expected_background.png
fig.write_image("../plots/expected_background.png", scale=2)